# Next-Generation File Formats (NGFF) Workshop

## Outline
* Notebook reminders
* Choosing our data & downloading from S3
* Converting your data to OME-NGFF
* Publishing your data with S3

## Notebook reminders
This notebook is somewhat unusual in that we use a lot of command-line tools. Each of the lines beginning with a exclamatino mark (`!`) is run in a terminal. You can perform those actions on your own system _without_ a Jupyter notebook. For that you will need to install the correct tools.

In [4]:
!conda info --envs

# conda environments:
#
base                     /srv/conda
notebook              *  /srv/conda/envs/notebook



In [7]:
!cat binder/environment.yml

channels:
  - ome
  - conda-forge
  - joshmoore
  - defaults
dependencies:
  - python == 3.9.0
  - napari
  - ipython
  - wheel
  - requests
  - s3fs
  - scikit-image
  - scipy
  - xarray
  - zarr >= 2.4.0
  - bioformats2raw
  - go-mc
  - pip
  - pip:
      - PyQT5==5.13.1
      - ome-zarr
      - awscli


### Other options
 * repo2docker
 * mybinder

## Data from S3

In [1]:
! pip install omero-cli-zarr

     |████████████████████████████████| 2.8 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 1.9 MB/s eta 0:00:01
  Created wheel for zeroc-ice: filename=zeroc_ice-3.6.5-cp39-cp39-linux_x86_64.whl size=2898266 sha256=689ef1919483962270daa45860a23ab5696c2e81e82426ea358d31e46025158b
  Stored in directory: /home/jamoore/.cache/pip/wheels/87/b7/37/f0246f397dcdadd36dbd6f01f9855c5f71c8087fe358ad5076
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=94174c7722badae3ecc99aba7eab0505a3e948eae1fec92bc5ab9dd5dc45718c
  Stored in directory: /home/jamoore/.cache/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built zeroc-ice future


In [ ]:
omero zarr export Image:6001240

# Cloud Basics

There are a number of different types of cloud storage and there are a number of tools that you can use to access your cloud storage, but here we're going to focus on a single one `mc`.

`mc` is provided by the minio project and is described as "a modern alternative to UNIX commands like ls, cat, cp, mirror, diff, find etc." The quickstart guide can be found under https://docs.minio.io/docs/minio-client-quickstart-guide.html For our purposes we'll focus on how to use it to upload and manage data in S3.

The minio project provides a safe space for you to learn about S3: https://play.minio.io:9000/minio/ You can find the access information by using the `mc` command.

## Setup

In [1]:
!mc config host list play

play
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto



 * "AccessKey" is basically a user name.
 * "SecretKey" is basically a password. 
 * The URL is our "endpoint", which differentiates it from the S3 servers provided by Amazon.

You can log in to the webpage and explore what the many other users have upload at https://play.minio.io:9000/minio/

The other two important concepts are:
 * "buckets" which is roughly like a shared namespace with permissions
 * and "keys" which will get to in a second.

For our purposes, we'll be using a single "bucket" `i2k2020`. Here we create it just in case it's been deleted in the background.

In [2]:
!mc mb --ignore-existing play/i2k2020

Bucket created successfully `play/i2k2020`.


In [3]:
!mc policy set public play/i2k2020

Access permission for `play/i2k2020` is set to `public`


We've now made our bucket public, anyone can access the files contained in it remotely.

We'll start by uploading a simple file to the bucket:

## Your first upload

In [4]:
!mc cp hello.txt play/i2k2020/hello.txt

hello.txt:     14 B / 14 B  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  18 B/s 0s

Now you can see list the contents of the bucket and see that it's there. Note that since other people are working on the same bucket, it may look have more or less contents each time you run this command.

In [5]:
!mc ls play/i2k2020/

[2020-12-01 11:37:29 CET]    14B hello.txt
[2020-11-30 21:20:24 CET]    14B josh.txt
[2020-12-01 11:38:21 CET]     0B gif.zarr/


Now visit https://play.minio.io:9000/i2k2020/hello.txt to see what you uploaded with the key of `hello.txt`. It's no longer a file. It's now in **object storage**.

## Content type (i.e. metadata)

We can also upload HTML:

In [6]:
!mc cp hello.html play/i2k2020/hello.html

hello.html:    46 B / 46 B  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  65 B/s 0s

which will render under https://play.minio.io:9000/i2k2020/hello.html However, if the name doesn't match the contents, the file will be downloaded rather than opened:

In [9]:
!mc cp hello.json play/i2k2020/hello.data

hello.json:    25 B / 25 B  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  34 B/s 0s

it may be downloaded: https://play.minio.io:9000/i2k2020/hello.data rather than shown. To fix that, you can add a content type:

In [10]:
!mc cp --attr=Content-Type=text/plain hello.txt play/i2k2020/hello.txt

hello.txt:     14 B / 14 B  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  25 B/s 0s

## Naming (i.e. keys)

Another important distinction to filesystems is that though it looks like hello is in a directory, you should really think of the entire string after the bucket just as a "key".

In [13]:
!mc rm play/i2k2020/hello.txt

Removing `play/i2k2020/hello.txt`.


In [14]:
!mc cp hello.txt play/i2k2020/josh.txt

hello.txt:     14 B / 14 B  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  25 B/s 0s

![object storage comparison](https://mk0openioo80ctbhsnje.kinstacdn.com/wp-content/uploads/2019/09/oio-block-files-object-storage-compared.png)

*from https://www.openio.io/blog/block-file-object-storage-evolution-computer-storage-systems*

# Conversion Tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="blog-2019-12-converting-whole-slide-images.jpg" style="height:300px" />



The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.

In [1]:
%%time
!bioformats2raw --help

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--version] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-directory=<memoDirectory>]
                    [--pyramid-name=<pyramidName>] [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-st

## Required OME-Zarr options
Two of the options are currently necessary to produce OME-Zarr data:
```
      --file_type=<fileType>
                     Tile file extension: n5, zarr (default: n5) [Can break
                       compatibility with raw2ometiff]
```
and
```
     --dimension-order=<dimensionOrder>
                     Override the input file dimension order in the output file
                       [Can break compatibility with raw2ometiff] (XYZCT,
                       XYZTC, XYCTZ, XYCZT, XYTCZ, XYTZC)
```

`--file_type` which produces Zarr output rather than N5 as the intermediate format. If we additionally pass the `--dimension-order` argument, then the intermediate result can be used directly by the ome-zarr library.

In [2]:
%%time
!bioformats2raw i2k2020.gif $PWD --file_type=zarr --dimension-order=XYZCT --scale-format-string=gif.zarr/%d

2020-12-01 12:25:46,540 [main] INFO  c.g.bioformats2raw.Converter - Output will be incompatible with raw2ometiff (pyramidName: data.zarr, scaleFormatString: gif.zarr/%d)
2020-12-01 12:25:47,075 [main] INFO  loci.formats.ImageReader - GIFReader initializing i2k2020.gif
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Verifying GIF format
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Reading dimensions
2020-12-01 12:25:47,075 [main] INFO  loci.formats.FormatHandler - Reading data blocks
2020-12-01 12:25:47,082 [main] INFO  loci.formats.FormatHandler - Populating metadata
2020-12-01 12:25:47,295 [main] INFO  c.g.bioformats2raw.Converter - Using 1 pyramid resolutions
2020-12-01 12:25:47,295 [main] INFO  c.g.bioformats2raw.Converter - Preparing to write pyramid sizeX 128 (tileWidth: 1024) sizeY 128 (tileWidth: 1024) imageCount 8
2020-12-01 12:25:47,559 [main] WARN  c.g.bioformats2raw.Converter - Reducing active tileWidth to 128
2020-12-01 12:25:47,559 [m

In [3]:
!ls *.ome.xml data.zarr

METADATA.ome.xml

data.zarr:
gif.zarr


## Moving to the cloud

You can then move the generated output to S3

In [4]:
!mc cp --recursive data.zarr/gif.zarr/ play/i2k2020/gif.zarr/

...5.0.0.0.0:  4.19 KiB / 4.19 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  5.10 KiB/s 0s

You can see your image under http://hms-dbmi.github.io/vizarr?source=https://play.minio.io:9000/i2k2020/gif.zarr

In [5]:
!mc cat play/i2k2020/gif.zarr/.zattrs

{"multiscales":[{"datasets":[{"path":"0"}],"version":"0.1"}]}

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.